# Setting up

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ta
import lightgbm as lgb
# from fastai import *
# from fastai.tabular import *
from tqdm import tqdm_notebook
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoLarsCV
from rolling import RollingWindowSplit
from sklearn.metrics import r2_score as r2d2
from joblib import dump, load
from datetime import datetime, timedelta

%matplotlib inline
%load_ext line_profiler
sns.set(style = "whitegrid")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
# %%time
# path = 'D://Coding//XTX Forecasting Challenge//data-training.csv'
# df = pd.read_csv(path)

In [3]:
path = 'D://Coding//XTX Forecasting Challenge//data-training.file'
df = pd.read_feather(path, use_threads=8)
df = df.astype('float32')
df.fillna(0, inplace=True)

In [4]:
askRateList = ['askRate' + str(i) for i in range(0,15)]
askSizeList = ['askSize' + str(i) for i in range(0,15)]
bidRateList = ['bidRate' + str(i) for i in range(0,15)]
bidSizeList = ['bidSize' + str(i) for i in range(0,15)]

In [5]:
core = [*askRateList, *askSizeList, *bidRateList, *bidSizeList]

In [6]:
cols = ['others_dr', 'others_dlr', 'others_cr']

# Exploratory Data Analysis

In [7]:
# # Figuring out what [y] is
# # y(t) is midRate(t+87) - midRate(t), clipped to (-5.5)
# df['expectedY'] = df.midRate.diff(87).shift(-87).clip(-5,5)

# Batch prediction

In [8]:
# the public leaderboard set should have 150k observations: they check running time of 10k in 1h and max 15h

In [9]:
# 70-30 train-valid and test split
def train_valid_test_split(df):
    valid_start = int(0.80*len(df))
    test_start = int(0.95*len(df))
    train_df = df[:valid_start].copy()
    valid_df = df[valid_start:test_start].copy()
    test_df = df[test_start:].copy()
    return train_df, valid_df, test_df
train_df, valid_df, test_df = train_valid_test_split(df)

In [10]:
def create_limited_features_orig(df, num):
    # limited features
    df['midRate'] = (df.bidRate0 + df.askRate0) / 2 # necessary for ohlc
    df['bidAskVol'] = df.bidSize0 + df.askSize0 # necessary for ohlc
    df['OIR'] = (df.bidSize0 - df.askSize0)/(df.bidSize0 + df.askSize0)
    df['totalAskVol'] = df[askSizeList].sum(axis=1)
    df['totalBidVol'] = df[bidSizeList].sum(axis=1)
    df['OIR_total'] = (df.totalBidVol - df.totalAskVol)/(df.totalBidVol + df.totalAskVol)
    
    df['spread'] = df.askRate0 - df.bidRate0
    df['bidAskVol'] = df.askSize0 + df.bidSize0
    df['vwaBid'] = np.einsum('ij,ji->i', df[bidRateList], df[bidSizeList].T) / df[bidSizeList].sum(axis=1)
    df['vwaAsk'] = np.einsum('ij,ji->i', df[askRateList], df[askSizeList].T) / df[askSizeList].sum(axis=1)
    df['vwaBidDMid'] = df.midRate - df.vwaBid
    df['vwaAskDMid'] = df.vwaAsk - df.midRate
    df['diff_vwaBidAskDMid'] = df.vwaAskDMid - df.vwaBidDMid
    
    # resample features
    df['time'] = pd.date_range(start='1/1/1970', periods=len(df), freq='T')
    df.set_index('time', inplace=True)
    df_mid = df.midRate.resample(str(num)+'Min').ohlc()
    df_mid['vol'] = df.bidAskVol.resample(str(num)+'Min').mean()
    def create_ohlc_features(df_mid, open_, high_, low_, close_, vol_):
        df_mid = ta.add_others_ta(df_mid, close_)
        return df_mid
    df_mid = create_ohlc_features(df_mid, 'open', 'high', 'low', 'close', 'vol')
    df = df.join(df_mid[df_mid.columns.drop(['open', 'high', 'low', 'close', 'vol'])]).ffill().astype('float32')
    df.fillna(0, inplace=True)
    return df, df_mid

In [11]:
rlcv = RollingWindowSplit(n_splits=3, compatible=True)

In [12]:
# takes 15s min
limited_train, df_mid_train = create_limited_features_orig(train_df, 120)
limited_valid, df_mid_valid = create_limited_features_orig(valid_df, 120)

In [13]:
def standardise(limited_train, scaler):
    x_train = limited_train.replace([np.inf, -np.inf], np.nan).fillna(0).drop('y', axis=1).values
    y_train = limited_train.y.values
    if scaler=='train':
        scaler = StandardScaler()
        scaler.fit(x_train)
    x_scaled_train = scaler.transform(x_train)
    return x_scaled_train, y_train, scaler

In [14]:
# takes 10s
x_scaled_train, y_train, scaler = standardise(limited_train, scaler='train')
x_scaled_valid, y_valid, scaler = standardise(limited_valid, scaler=scaler)

In [15]:
fixed_params = {'boosting_type': 'gbdt',
                'nthread': 4,
                'random_state': 42,
                'metric': 'rmse'}

grid_params = {'learning_rate': [0.02, 0.03, 0.04, 0.05],
                'num_leaves': [120,140,160,180,200],
                'max_bin': [60,70,80,90,100],
                'max_depth' : [0,1,2,3,4,5],
                'colsample_bytree' : [0.7,0.8,0.9,1],
                'subsample' : [0.3,0.5,0.7],
                'min_split_gain' : [0.6, 0.7, 0.8, 0.9],
                'min_data_in_leaf':[30,40,50,60],
                'reg_alpha': [0.1,0.3,0.5,0.7,1],
                'reg_lambda': [0.1,1,5,10],
              }

In [16]:
''' RandomSearch for optimal LGBM parameters '''
# clf = lgb.LGBMRegressor(**fixed_params, eval_set=(x_scaled_valid, y_valid), eval_metric='l2')
# grid3 = RandomizedSearchCV(clf, grid_params, verbose=1, cv=rlcv, n_jobs = -1, n_iter=30)
# grid3.fit(x_scaled_train, y_train)

' RandomSearch for optimal LGBM parameters '

In [17]:
best_params = {'subsample': 0.7, 'reg_lambda': 0.1, 'reg_alpha': 0.3,
                'num_leaves': 200, 'min_split_gain': 0.9, 'min_data_in_leaf': 60,
                'max_depth': 3, 'max_bin': 60, 'learning_rate': 0.02, 'colsample_bytree': 0.8}

In [18]:
# takes 41s min
lgbm = lgb.LGBMRegressor(**fixed_params, **best_params, eval_set=(x_scaled_valid, y_valid), eval_metric='l2').fit(x_scaled_train, y_train)

In [27]:
dump(scaler, 'scaler120.joblib')
dump(lgbm, 'lgbm120.joblib')
# scaler = load('scaler110.joblib')
# lgbm = load('lgbm110.joblib')

['lgbm120.joblib']

In [20]:
''' We want to try different ohlc window sizes. '''

' We want to try different ohlc window sizes. '

In [21]:
'''
Trying 2:	0.0188, 0.0205, 0.0190
Trying 5:	0.0233, 0.0254, 0.0245
Trying 8:	0.0294, 0.0321, 0.0317
Trying 11:	0.0363, 0.0395, 0.0380
Trying 14:	0.0440, 0.0461, 0.0464
Trying 17:	0.0514, 0.0535, 0.0508
Trying 20:	0.0585, 0.0606, 0.0614
Trying 23:	0.0665, 0.0695, 0.0659
Trying 26:	0.0746, 0.0773, 0.0745
Trying 29:	0.0833, 0.0861, 0.0826
Trying 32:	0.0916, 0.0900, 0.0902
Trying 35:	0.0989, 0.1000, 0.0979
Trying 38:	0.1075, 0.1050, 0.1090
Trying 41:	0.1134, 0.1107, 0.1231
Trying 44:	0.1208, 0.1220, 0.1297
Trying 47:	0.1317, 0.1267, 0.1346
Trying 50:	0.1372, 0.1378, 0.1370
Trying 53:	0.1478, 0.1440, 0.1552
Trying 56:	0.1529, 0.1496, 0.1559
Trying 59:	0.1602, 0.1608, 0.1679
Trying 62:	0.1672, 0.1639, 0.1766
Trying 65:	0.1800, 0.1795, 0.1828
Trying 68:	0.1820, 0.1820, 0.1692
Trying 71:	0.1937, 0.2007, 0.1959
Trying 74:	0.2032, 0.2005, 0.2216
Trying 77:	0.2076, 0.2088, 0.2094
Trying 80:	0.2149, 0.2129, 0.2235
Trying 83:	0.2271, 0.2210, 0.2297
Trying 86:	0.2338, 0.2277, 0.2343
Trying 89:	0.2403, 0.2400, 0.2647
Trying 92:	0.2510, 0.2408, 0.2570 <-- maybe consider 90 as well
Trying 95:	0.2594, 0.2500, 0.2708
Trying 98:	0.2649, 0.2538, 0.2646
Trying 101:	0.2631, 0.2634, 0.2577
Trying 104:	0.2727, 0.2718, 0.2954
Trying 107:	0.2699, 0.2803, 0.2831
Trying 110:	0.2775, 0.2743, 0.3013 <-- memory constraints suggest we use 110 (actually..idk)
Trying 113:	0.2778, 0.2718, 0.2952
Trying 116:	0.2799, 0.2760, 0.2854
Trying 119:	0.2781, 0.2704, 0.2955 <-- we use 120 to avoid a misalignment problem in time
Trying 122:	0.2799, 0.2777, 0.2891
Trying 125:	0.2843, 0.2844, 0.2864
Trying 128:	0.2890, 0.2738, 0.3027
Trying 131:	0.2874, 0.2843, 0.2938
Trying 134:	0.2833, 0.2774, 0.2956
Trying 137:	0.2858, 0.2844, 0.2822
Trying 140:	0.2828, 0.2754, 0.2930
Trying 143:	0.2815, 0.2752, 0.3085
Trying 146:	0.2751, 0.2774, 0.3054
Trying 149:	0.2824, 0.2827, 0.2950

Trying 155:	0.2763, 0.2797, 0.2963
Trying 160:	0.2799, 0.2643, 0.2847
Trying 165:	0.2705, 0.2753, 0.2890
Trying 170:	0.2752, 0.2643, 0.2683
Trying 175:	0.2759, 0.2630, 0.2779
Trying 180:	0.2670, 0.2544, 0.2736
Trying 185:	0.2639, 0.2499, 0.2638
'''

'\nTrying 2:\t0.0188, 0.0205, 0.0190\nTrying 5:\t0.0233, 0.0254, 0.0245\nTrying 8:\t0.0294, 0.0321, 0.0317\nTrying 11:\t0.0363, 0.0395, 0.0380\nTrying 14:\t0.0440, 0.0461, 0.0464\nTrying 17:\t0.0514, 0.0535, 0.0508\nTrying 20:\t0.0585, 0.0606, 0.0614\nTrying 23:\t0.0665, 0.0695, 0.0659\nTrying 26:\t0.0746, 0.0773, 0.0745\nTrying 29:\t0.0833, 0.0861, 0.0826\nTrying 32:\t0.0916, 0.0900, 0.0902\nTrying 35:\t0.0989, 0.1000, 0.0979\nTrying 38:\t0.1075, 0.1050, 0.1090\nTrying 41:\t0.1134, 0.1107, 0.1231\nTrying 44:\t0.1208, 0.1220, 0.1297\nTrying 47:\t0.1317, 0.1267, 0.1346\nTrying 50:\t0.1372, 0.1378, 0.1370\nTrying 53:\t0.1478, 0.1440, 0.1552\nTrying 56:\t0.1529, 0.1496, 0.1559\nTrying 59:\t0.1602, 0.1608, 0.1679\nTrying 62:\t0.1672, 0.1639, 0.1766\nTrying 65:\t0.1800, 0.1795, 0.1828\nTrying 68:\t0.1820, 0.1820, 0.1692\nTrying 71:\t0.1937, 0.2007, 0.1959\nTrying 74:\t0.2032, 0.2005, 0.2216\nTrying 77:\t0.2076, 0.2088, 0.2094\nTrying 80:\t0.2149, 0.2129, 0.2235\nTrying 83:\t0.2271, 0.2210, 

In [22]:
def score(model, scaler, number, x_scaled_train, x_scaled_valid, y_train, y_valid):
    train_score = model.score(x_scaled_train, y_train)
    predictions_valid = model.predict(x_scaled_valid)
    valid_score = r2d2(y_valid, predictions_valid)
    limited_test, df_mid_test = create_limited_features_orig(test_df, number)
    x_scaled_test, y_test, scaler = standardise(limited_test, scaler=scaler)
    predictions_test = model.predict(x_scaled_test)
    test_score = r2d2(y_test, predictions_test)
    print(f'{train_score:.4f}, {valid_score:.4f}, {test_score:.4f}')
    return predictions_test, limited_test, df_mid_test

In [23]:
# nums = list(np.arange(255,500,10))
# def test_various_windows():
#     for number in tqdm_notebook(nums):
#         train = create_limited_features_orig(train_df, number)
#         valid = create_limited_features_orig(valid_df, number)
#         x_scaled_train, y_train, scaler = standardise(train, scaler='train')
#         x_scaled_valid, y_valid, scaler = standardise(valid, scaler=scaler)
#         lgbm = lgb.LGBMRegressor(**fixed_params, **best_params, eval_set=(x_scaled_valid, y_valid),
#                                  eval_metric='l2').fit(x_scaled_train, y_train)
#         print(f'Trying {number}:', end='\t')
#         predictions, limited_test = score(lgbm, scaler, number, x_scaled_train, x_scaled_valid,
#                                           y_train, y_valid)
# test_various_windows()

In [24]:
''' -1000 has scores of (0.0457, 0.0481, -0.1535) '''

' -1000 has scores of (0.0457, 0.0481, -0.1535) '

In [25]:
'''Shifted features are super important!'''
'''Don't drop original variables!'''
'''Train on entire dataset!'''

'Train on entire dataset!'

In [26]:
predictions, limited_test, df_mid_test = score(lgbm, scaler, 120, x_scaled_train, x_scaled_valid, y_train, y_valid)

0.2736, 0.2683, 0.3011


In [ ]:
sns.set(style='whitegrid')

In [ ]:
plt.subplots(1,1,figsize=(15,8))
plt.plot(predictions[:10000])
plt.plot(limited_test.y.values[:10000])
plt.legend(['predictions','true y'])

In [ ]:
# takes 1 min
predictions, limited_test = score(lgbm, scaler) # 0.95 most recent
''' 0.95 has scores of (0.0456, 0.0489, 0.0451)'''

In [ ]:
a = lgbm.feature_importances_; a

In [ ]:
limited_train.columns

In [ ]:
np.where(a<10,1,0).astype(bool)

In [ ]:
cols = limited_train.columns.drop('y')

In [ ]:
'''You should chuck the variables that meet this condition!'''
cols.values[np.where(a<10,1,0).astype(bool)]

In [ ]:
cols.values[np.where(a>100,1,0).astype(bool)]

In [ ]:
predictions, limited_test, df_mid_test = score(lasso)

# Fast.ai

In [ ]:
dep_var = 'y'
procs = [FillMissing, Normalize]

In [ ]:
# fillmissing replaces with median // fill with zero could be better

In [ ]:
# use a subset of training data
train_df = train_df[:int(5e5)]

In [ ]:
# takes 7s on 500k rows, 30s on full df
test = TabularList.from_df(test_df, procs=procs)
data = (TabularList.from_df(df = train_df, path='.', cont_names = df.columns.drop('y'), procs=procs)
                            .split_by_idx(valid_idx=range(int(0.50*len(train_df)),int(len(train_df))))
                            .label_from_df(cols=dep_var)
                            .add_test(test, label=0)
                            .databunch())

In [ ]:
learn = tabular_learner(data, layers=[500,200], metrics=[mean_absolute_error, r2_score], ps=[0.001,0.01])

In [ ]:
learn.model

In [ ]:
learn.lr_find(end_lr=1e1)

In [ ]:
sns.set(style = "whitegrid")

In [ ]:
learn.recorder.plot()

In [ ]:
# model above has already diverged, we will restart.

In [ ]:
# create pytorch resnet model






In [ ]:
# do some learning







In [ ]:
# 2m for 1 cycle
learn.fit_one_cycle(1, 5e-4, wd=0.01)

In [ ]:
learn.recorder.plot_lr(show_moms=True)

In [ ]:
learn.save('new_fastai')

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.show_results()

In [ ]:
Learner.predict??

In [ ]:
df.iloc[:100].values.shape

In [ ]:
learn.predict(df.iloc[int(8.1e5)])

In [ ]:
df.y.iloc[int(8.1e5)]

In [ ]:
preds = learn.get_preds()

In [ ]:
Learner.get_preds??

# Submission testing

In [28]:
def get_next_data_as_df(test_df, iteration, length):
    return pd.DataFrame([test_df.head(length).iloc[iteration][:60].values])

In [29]:
'''
This function takes in a DataFrame row [df] of shape (1,60) and adds 10 cross-sectional features.
Returns a DataFrame of shape (1,70).
'''
def create_limited_features(df):
    df.columns = [*askRateList, *askSizeList, *bidRateList, *bidSizeList]
    # limited features
    df['midRate'] = (df.bidRate0 + df.askRate0) / 2 # necessary for ohlc
    df['bidAskVol'] = df.bidSize0 + df.askSize0 # necessary for ohlc
    df['OIR'] = (df.bidSize0 - df.askSize0)/(df.bidSize0 + df.askSize0)
    df['totalAskVol'] = df[askSizeList].sum(axis=1)
    df['totalBidVol'] = df[bidSizeList].sum(axis=1)
    df['OIR_total'] = (df.totalBidVol - df.totalAskVol)/(df.totalBidVol + df.totalAskVol)
    
    df['spread'] = df.askRate0 - df.bidRate0
    df['bidAskVol'] = df.askSize0 + df.bidSize0
    df['vwaBid'] = np.einsum('ij,ji->i', df[bidRateList], df[bidSizeList].T) / df[bidSizeList].sum(axis=1)
    df['vwaAsk'] = np.einsum('ij,ji->i', df[askRateList], df[askSizeList].T) / df[askSizeList].sum(axis=1)
    df['vwaBidDMid'] = df.midRate - df.vwaBid
    df['vwaAskDMid'] = df.vwaAsk - df.midRate
    df['diff_vwaBidAskDMid'] = df.vwaAskDMid - df.vwaBidDMid
    return df

In [30]:
''' Appends to [massive_df]=(many, >70) the DataFrame row [row]=(1,70). '''
def append_to_df(massive_df, row):
    try: row.index = [massive_df.index[-1] + timedelta(minutes=1)]
    except IndexError: row.index = [datetime(1970,1,1)]
    return massive_df.append(row, sort=False)

In [31]:
''' Adds time-dependent features. All features that use shift/diff must come here. '''
def add_time_features(df, num):
    return df[-num:]

In [32]:
''' Creates more features based on resampled OHLC data. '''
def add_resample_features(massive_df, resampled_df, num):
    resampled_df_size = 15
    leftovers = (massive_df.index[-1].value // 6e10 + 1) % num
    def create_ohlc_features(df_mid, open_, high_, low_, close_, vol_):
        df_mid = ta.add_others_ta(df_mid, close_)
        return df_mid
    row_ohlcv = massive_df.midRate.resample(str(num)+'Min').ohlc().tail(1)
    row_ohlcv['vol'] = massive_df.bidAskVol.resample(str(num)+'Min').mean()
    full_resampled = resampled_df.append(row_ohlcv, sort=False)
    full_resampled = create_ohlc_features(full_resampled, 'open', 'high', 'low', 'close', 'vol')
    if leftovers == 0:
        resampled_df = resampled_df.append(row_ohlcv, sort=False).tail(resampled_df_size) # this adds a completed row_ohlcv
    try: massive_df.drop(cols, axis=1, inplace=True)
    except KeyError: pass
    massive_df = massive_df.join(full_resampled[cols])
    massive_df = massive_df.ffill().astype('float32') # no fillna because leaving rsi as nan is probably appropriate
    return massive_df, resampled_df, full_resampled # full_resampled only for debug

In [33]:
''' This function takes in all features and makes a bounded prediction. '''
def get_prediction(data, model):
    X = data.replace([np.inf, -np.inf], np.nan).values
    X_scaled = scaler.transform(X)
    return np.clip(model.predict(np.atleast_2d(X_scaled)), -5, 5)[0]

In [34]:
''' true_rlcvscore runs the submission functions on a test dataframe [test_df] taking the first [length] rows of [test_df].'''
def true_rlcvscore(test_df, model, length):
    num = 120
    massive_df, resampled_df = pd.DataFrame(), pd.DataFrame()
    predictions = []
    log_data, resampled_data_list, full_resampled_data_list, massive_df_list = pd.DataFrame(), [], [], []  # for debug
    for iteration in tqdm_notebook(range(length)):
        base_row = get_next_data_as_df(test_df, iteration, length)
        row = create_limited_features(base_row)
        massive_df = append_to_df(massive_df, row)
        massive_df = add_time_features(massive_df, num)
        massive_df, resampled_df, full_resampled = add_resample_features(massive_df, resampled_df, num)
        data = pd.DataFrame([massive_df.iloc[-1]])
        log_data_row = data.copy() # for debug
        prediction = get_prediction(data, model)
        predictions.append(prediction)
        log_data = log_data.append(log_data_row, sort=False) # for debug
        resampled_data_list.append(resampled_df) # for debug
        full_resampled_data_list.append(full_resampled) # for debug
        massive_df_list.append(massive_df) # for debug
    true_values = test_df.y.head(length)
    score = r2d2(true_values, predictions)
    print(f'{score:.4f}')
    return predictions, score, log_data, true_values, resampled_data_list, full_resampled_data_list, massive_df_list

In [35]:
# takes 2m30s for 1000
(test_predictions, test_score, log_data, true_values,
 resampled_data_list, full_resampled_data_list, massive_df_list) = true_rlcvscore(test_df, lgbm, 250)


-1.7188


In [ ]:
s1 = pd.Series(limited_test.head(2000).others_dr, name='batch')
s2 = pd.Series(log_data.others_dr, name='line-by-line')
vals = pd.concat([s1,s2],axis=1); vals
vals['difference'] = vals.batch - vals['line-by-line']

In [ ]:
plt.subplots(1,1,figsize=(15,8))
plt.plot(limited_test.head(2000).others_dr.values)
plt.plot(log_data.others_dlr.values)
plt.title('others_dlr', fontsize=20)
plt.legend(['batch','line-by-line'], fontsize=20)

In [ ]:
'''consider creating a model that takes in (1 to 120 rows) and predicts the final value. '''

In [ ]:
plt.subplots(1,1, figsize=(15,8))
plt.plot(vals.difference.values)

In [ ]:
''' the values are almost the same i.e. 3rd decimal place onwards. but the plot above says its from the first decimal place onwards'''

In [ ]:
'''full_resampled is wrong --> log_data has wrong values
'''

In [ ]:
''' The most important thing now is to reconcile the dataframes in submission and in batch prediction. '''

In [ ]:
plt.subplots(1,1,figsize=(15,8))
plt.plot(test_predictions)
plt.plot(true_values.values)
plt.legend(['predictions', 'y'])

In [ ]:
# %lprun -f true_rlcvscore test_predictions, test_score = true_rlcvscore(test_df, 100)